# GBDT training on RTX 3090 (ordered, quiet, early-stopping)

This notebook trains XGBoost (GPU) and LightGBM (GPU if available) on `dataset_pruned.csv` with the paper's hyperparameters, proper preprocessing, and early stopping to avoid long runs. Logs are silenced.


In [1]:
import os, json, warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings('ignore')

DATA_PATH = 'dataset_pruned.csv'
LABEL_COL = 'weight_kg'
ID_COLS = ['Chicken_ID', 'Image_ID']

assert os.path.exists(DATA_PATH), f"Missing {DATA_PATH}"

df = pd.read_csv(DATA_PATH)
print(f"Loaded: {df.shape}")

# Drop rows with NaN label
before = df.shape[0]
df = df[~df[LABEL_COL].isna()].reset_index(drop=True)
print(f"Dropped {before - df.shape[0]} rows with NaN label")

# Coerce features to numeric and clean inf
feature_cols = [c for c in df.columns if c not in (ID_COLS + [LABEL_COL])]
for c in feature_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df[feature_cols] = df[feature_cols].replace([np.inf, -np.inf], np.nan)

# Split
X = df[feature_cols]
y = df[LABEL_COL].values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.30, random_state=42, shuffle=True)

# Impute + scale (fit on train only)
imputer = SimpleImputer(strategy='median')
X_tr_imp = imputer.fit_transform(X_tr)
X_te_imp = imputer.transform(X_te)

scaler = StandardScaler(with_mean=True, with_std=True)
X_train = scaler.fit_transform(X_tr_imp)
X_test = scaler.transform(X_te_imp)

print("Train/Test:", X_train.shape, X_test.shape)


Loaded: (1492, 2076)
Dropped 1 rows with NaN label
Train/Test: (1043, 2073) (448, 2073)


In [2]:
# XGBoost (GPU)
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=2000,
    learning_rate=0.1,
    max_depth=5,
    objective='reg:squarederror',
    min_child_weight=1,
    max_delta_step=0,
    subsample=0.8,
    colsample_bytree=0.7,
    colsample_bylevel=1.0,
    reg_alpha=0.0,
    reg_lambda=1.0,
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    gpu_id=0,
    random_state=42,
    n_jobs=0,
)

xgb.fit(X_train, y_tr, eval_set=[(X_test, y_te)], verbose=False)
print("XGB trained (GPU)")


XGB trained (GPU)


In [4]:
# LightGBM (GPU if available) with early stopping and quiet logs
import lightgbm as lgb

params = dict(
    n_estimators=4000,
    learning_rate=0.1,
    num_leaves=15,
    max_depth=5,
    min_child_samples=15,
    min_child_weight=0.01,
    subsample=0.8,
    colsample_bytree=1.0,
    objective='regression',
    n_jobs=-1,
    verbosity=-1,
)

# Try GPU
use_gpu = False
try:
    params_gpu = {**params, 'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0}
    lgbm = lgb.LGBMRegressor(**params_gpu)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )
    use_gpu = True
except Exception as e:
    lgbm = lgb.LGBMRegressor(**params)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )

print(f"LGBM trained (GPU={use_gpu})")


LGBM trained (GPU=True)


In [ ]:
# LightGBM (GPU if available) with early stopping and quiet logs
import lightgbm as lgb

params = dict(
    n_estimators=4000,
    learning_rate=0.1,
    num_leaves=15,
    max_depth=5,
    min_child_samples=15,
    min_child_weight=0.01,
    subsample=0.8,
    colsample_bytree=1.0,
    objective='regression',
    n_jobs=-1,
    verbosity=-1,
)

# Try GPU
use_gpu = False
try:
    params_gpu = {**params, 'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0}
    lgbm = lgb.LGBMRegressor(**params_gpu)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )
    use_gpu = True
except Exception as e:
    lgbm = lgb.LGBMRegressor(**params)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )

print(f"LGBM trained (GPU={use_gpu})")


LGBM trained (GPU=True)


In [5]:
# Evaluate
import numpy as np

def evaluate(model, X_te, y_te, name):
    pred = model.predict(X_te)
    mae = mean_absolute_error(y_te, pred)
    mse = mean_squared_error(y_te, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_te, pred)
    print(f"{name} -> MAE: {mae:.6f}  MSE: {mse:.6f}  RMSE: {rmse:.6f}  R2: {r2:.6f}")
    return dict(model=name, MAE=mae, MSE=mse, RMSE=rmse, R2=r2)

results = []
results.append(evaluate(lgbm, X_test, y_te, f"LGBM_GPU={use_gpu}"))
results.append(evaluate(xgb, X_test, y_te, 'XGB_GPU'))

import pandas as pd
pd.DataFrame(results)


LGBM_GPU=True -> MAE: 0.068438  MSE: 0.009456  RMSE: 0.097241  R2: 0.870543
XGB_GPU -> MAE: 0.069832  MSE: 0.010160  RMSE: 0.100797  R2: 0.860904


,model,MAE,MSE,RMSE,R2
0,LGBM_GPU=True,0.068438,0.009456,0.097241,0.870543
1,XGB_GPU,0.069832,0.010160,0.100797,0.860904


In [ ]:
# LightGBM (GPU if available) with early stopping and quiet logs
import lightgbm as lgb

params = dict(
    n_estimators=4000,
    learning_rate=0.1,
    num_leaves=15,
    max_depth=5,
    min_child_samples=15,
    min_child_weight=0.01,
    subsample=0.8,
    colsample_bytree=1.0,
    objective='regression',
    n_jobs=-1,
    verbosity=-1,
)

# Try GPU
use_gpu = False
try:
    params_gpu = {**params, 'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0}
    lgbm = lgb.LGBMRegressor(**params_gpu)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )
    use_gpu = True
except Exception as e:
    lgbm = lgb.LGBMRegressor(**params)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )

print(f"LGBM trained (GPU={use_gpu})")


LGBM trained (GPU=True)


In [ ]:
# LightGBM (GPU if available) with early stopping and quiet logs
import lightgbm as lgb

params = dict(
    n_estimators=4000,
    learning_rate=0.1,
    num_leaves=15,
    max_depth=5,
    min_child_samples=15,
    min_child_weight=0.01,
    subsample=0.8,
    colsample_bytree=1.0,
    objective='regression',
    n_jobs=-1,
    verbosity=-1,
)

# Try GPU
use_gpu = False
try:
    params_gpu = {**params, 'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0}
    lgbm = lgb.LGBMRegressor(**params_gpu)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )
    use_gpu = True
except Exception as e:
    lgbm = lgb.LGBMRegressor(**params)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )

print(f"LGBM trained (GPU={use_gpu})")


LGBM trained (GPU=True)


In [ ]:
# LightGBM (GPU if available) with early stopping and quiet logs
import lightgbm as lgb

params = dict(
    n_estimators=4000,
    learning_rate=0.1,
    num_leaves=15,
    max_depth=5,
    min_child_samples=15,
    min_child_weight=0.01,
    subsample=0.8,
    colsample_bytree=1.0,
    objective='regression',
    n_jobs=-1,
    verbosity=-1,
)

# Try GPU
use_gpu = False
try:
    params_gpu = {**params, 'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0}
    lgbm = lgb.LGBMRegressor(**params_gpu)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )
    use_gpu = True
except Exception as e:
    lgbm = lgb.LGBMRegressor(**params)
    lgbm.fit(
        X_train, y_tr,
        eval_set=[(X_test, y_te)],
        eval_metric='l1',
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )

print(f"LGBM trained (GPU={use_gpu})")


LGBM trained (GPU=True)


In [ ]:
# Save artifacts and details (timestamped folder)
import os, json, joblib, datetime

# Create timestamped run directory under saved_models/
ts = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
out_base = 'saved_models'
out_dir = os.path.join(out_base, f'run_{ts}')
os.makedirs(out_dir, exist_ok=True)

# Save artifacts
joblib.dump(imputer, os.path.join(out_dir, 'imputer.joblib'))
joblib.dump(scaler, os.path.join(out_dir, 'scaler.joblib'))
joblib.dump(lgbm, os.path.join(out_dir, 'lgbm_model.joblib'))
joblib.dump(xgb, os.path.join(out_dir, 'xgb_gpu_model.joblib'))

# Save metrics
with open(os.path.join(out_dir, 'metrics.json'), 'w') as f:
    json.dump(results, f, indent=2)

# Save run details
details = {
    'timestamp': datetime.datetime.now().isoformat(),
    'dataset': 'dataset_pruned.csv',
    'train_shape': X_train.shape,
    'test_shape': X_test.shape,
    'lgbm_used_gpu': bool(use_gpu),
    'lgbm_params': lgbm.get_params(),
    'xgb_params': xgb.get_params(),
    'output_dir': out_dir,
}
with open(os.path.join(out_dir, 'training_3090_details.txt'), 'w') as f:
    f.write(json.dumps(details, indent=2))

print(f"Saved to {out_dir}")


Saved to saved_models
